# Combinatorial Polyhedra and development of geometric Polyhedra in Sage

## Combinatorial Polyhedron


### Motivation

Backends compute the double description  
-> vertex-facet incidences

The following should be easy. Right?
- $f$-vector,
- the vertex graph,
- the facet graph,
- ($n$-dimensional) faces,
- simplicity/simpliciality,
- flag-$f$-vector,
- the face lattice.

Well, the face lattice can be huge.

In [2]:
P = polytopes.permutahedron(6)
%time P.face_lattice()

CPU times: user 15.4 s, sys: 2.48 ms, total: 15.4 s
Wall time: 15.4 s


Finite lattice containing 4684 elements with distinguished linear extension (use the .plot() method to plot)

Not long ago, the list was handled by the face lattice.

### Example:

Hirsch conjecture counterexample of Matschke, Santos, Weibel ([arXiv:1202.4701](https://arxiv.org/abs/1202.4701)).

### (Was) Impossible!

$20$-dimensional, $40$ facets, $36425$ vertices

`CombinatorialPolyhedron` is faster (not yet exposed):

In [13]:
P = polytopes.permutahedron(6)
%time P.combinatorial_polyhedron().face_lattice()

CPU times: user 492 ms, sys: 0 ns, total: 492 ms
Wall time: 504 ms


Finite lattice containing 4684 elements (use the .plot() method to plot)

But it also avoids the face lattice.

In [3]:
P = polytopes.permutahedron(6)
_ = P.incidence_matrix()
%time P.f_vector()

CPU times: user 117 ms, sys: 3.67 ms, total: 121 ms
Wall time: 123 ms


(1, 720, 1800, 1560, 540, 62, 1)

## Input of `CombinatorialPolyhedron`

- a polyhedron,
- a lattice polytopes,
- a cone,
- an incidence matrix,
- a list of facets, each facet given as list of vertices/rays/lines.

In [17]:
CombinatorialPolyhedron(cones.rearrangement(3, 8))  # new cone library in 9.1

A 8-dimensional combinatorial polyhedron with 56 facets

In [18]:
CombinatorialPolyhedron([[0,1,2], [0,1,3], [0,2,3], [1,2,3]])

A 3-dimensional combinatorial polyhedron with 4 facets

In [21]:
CombinatorialPolyhedron(Matrix([[0,1], [1,0]]))

A 1-dimensional combinatorial polyhedron with 2 facets

## The $f$-vector

Sage has the only memory efficient implementation

At least as fast as other implementations

Ongoing development (parallelization, some improvements)

### Benchmarks (1)

In [23]:
P = polytopes.simplex(24)
_ = P.incidence_matrix()
%time P.f_vector()

CPU times: user 2.43 s, sys: 11.9 ms, total: 2.44 s
Wall time: 2.43 s


(1, 25, 300, 2300, 12650, 53130, 177100, 480700, 1081575, 2042975, 3268760, 4457400, 5200300, 5200300, 4457400, 3268760, 2042975, 1081575, 480700, 177100, 53130, 12650, 2300, 300, 25, 1)

- more than 30 GB of RAM with `polymake`
- 40 seconds with `normaliz` (8 threads, 4 cores)

### Benchmarks (2)

In [17]:
P = polytopes.associahedron(['A', 10] , backend='normaliz')
_ = P.incidence_matrix()
%time P.f_vector()

CPU times: user 22.5 s, sys: 19.5 ms, total: 22.5 s
Wall time: 22.5 s


(1, 58786, 293930, 629850, 755820, 556920, 259896, 76440, 13650, 1365, 65, 1)

- 7 hours with `polymake` 
- 16 seconds with `normaliz` (8 threads on 4 cores).

Possible improvements for `sage`:
- sorting the vertices by significance (factor of $2$)
- goal: 1s

### Benchmarks (3)

In [16]:
P = polytopes.Birkhoff_polytope(5, backend='normaliz')
_ = P.incidence_matrix()
%time P.f_vector()

CPU times: user 538 ms, sys: 0 ns, total: 538 ms
Wall time: 538 ms


(1, 120, 5040, 50250, 233400, 631700, 1113700, 1367040, 1220550, 817150, 419225, 167200, 52120, 12600, 2300, 300, 25, 1)

In [17]:
import PyNormaliz
c = P._normaliz_cone
%time _ = PyNormaliz.NmzResult(c, "FVector")

CPU times: user 51.7 s, sys: 227 ms, total: 52 s
Wall time: 16.5 s


- 8 seconds with `normaliz` with 4 cores
- 6 minutes with `polymake`

### Benchmarks (4)

#### $25$-dimensional $6$-Birkhoff:

- 45 minutes with `sage`
- more than 30 GB RAM with `polymake` or `normaliz`

`sage` needs little memory:


In [1]:
start = get_memory_usage()
P = polytopes.Birkhoff_polytope(6)
_ = P.incidence_matrix()
intermediate = get_memory_usage()

it = P.face_generator()
for _ in range(10000):
    a = next(it)
end = get_memory_usage()
print("Total memory: ", end - start, "\nMemory of FaceIterator: ", end - intermediate)

Total memory:  48.3828125 
Memory of FaceIterator:  15.796875


## Vertex graph/facet graph

In [5]:
P = polytopes.hypercube(10)
C = P.combinatorial_polyhedron()
%time C.vertex_graph(names=False)

CPU times: user 106 ms, sys: 0 ns, total: 106 ms
Wall time: 106 ms


Graph on 1024 vertices (use the .plot() method to plot)

In [6]:
%time C.facet_graph(names=False)

CPU times: user 2.16 ms, sys: 0 ns, total: 2.16 ms
Wall time: 2.17 ms


Graph on 21 vertices (use the .plot() method to plot)

Vertex graph is exposed, but slow (naming vertices of a graph)

In [7]:
%time P.vertex_graph()

CPU times: user 3.56 s, sys: 3.91 ms, total: 3.57 s
Wall time: 3.56 s


Graph on 1024 vertices (use the .plot() method to plot)

## ($n$-)dimensional faces/face iterator

Main feature of `CombinatorialPolyhedron`

Algorithm: [arXiv:1905.01945](https://arxiv.org/abs/1905.01945)
(joint work with Christian Stump)

Exposed through `faces`, `face_generator` in `Polyhedron_base`

In [11]:
P = polytopes.hypercube(14, backend='field')
C = P.combinatorial_polyhedron()
it = C.face_iter(dimension=5)
%time f = next(it)

CPU times: user 107 µs, sys: 3 µs, total: 110 µs
Wall time: 113 µs


In [15]:
[f, f.ambient_V_indices()[:5], f.n_ambient_Hrepresentation(), f.ambient_Hrepresentation()[:2]]

[A 5-dimensional face of a 14-dimensional combinatorial polyhedron,
 (0, 512, 1024, 1536, 2048),
 9,
 (An inequality (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0) x + 1 >= 0,
  An inequality (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0) x + 1 >= 0)]

### All faces

In [6]:
C = polytopes.simplex().combinatorial_polyhedron()
for i in C.face_iter(): print(i)  # non-dual by default

A 2-dimensional face of a 3-dimensional combinatorial polyhedron
A 2-dimensional face of a 3-dimensional combinatorial polyhedron
A 2-dimensional face of a 3-dimensional combinatorial polyhedron
A 2-dimensional face of a 3-dimensional combinatorial polyhedron
A 1-dimensional face of a 3-dimensional combinatorial polyhedron
A 1-dimensional face of a 3-dimensional combinatorial polyhedron
A 1-dimensional face of a 3-dimensional combinatorial polyhedron
A 0-dimensional face of a 3-dimensional combinatorial polyhedron
A 0-dimensional face of a 3-dimensional combinatorial polyhedron
A 0-dimensional face of a 3-dimensional combinatorial polyhedron
A 1-dimensional face of a 3-dimensional combinatorial polyhedron
A 1-dimensional face of a 3-dimensional combinatorial polyhedron
A 0-dimensional face of a 3-dimensional combinatorial polyhedron
A 1-dimensional face of a 3-dimensional combinatorial polyhedron


In [17]:
C = polytopes.cross_polytope(3).combinatorial_polyhedron()
it = C.face_iter()  # dual mode by default
[next(it) for _ in range(11)]

[A 0-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 0-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 0-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 0-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 0-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 0-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 1-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 1-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 1-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 1-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron]

### Variations

#### Specify dual/non-dual

In [18]:
C = polytopes.cross_polytope(3).combinatorial_polyhedron()
it = C.face_iter(dual=False)
[next(it) for _ in range(10)]

[A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 1-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 1-dimensional face of a 3-dimensional combinatorial polyhedron]

#### Specify face dimension

In [20]:
C.face_iter(2)

Iterator over the 2-faces of a 3-dimensional combinatorial polyhedron

#### Ignore subfaces/supfaces

Save time by ignoring subfaces (supfaces in dual mode) of current face

Demonstration with details on [ask.sagemath.org/question/34485](https://ask.sagemath.org/question/34485/what-is-the-most-efficient-way-to-look-up-a-face-in-the-face-lattice-of-a-polyhedron/?answer=50965#post-id-50965)

## Simplicity/simpliciality

A polyhedron is $k$-simplicial if every $k$-face is a simplex.
A polyhedron is $k$-simple, if its dual is $k$-simplicial.

In [16]:
P = polytopes.hypersimplex(12,5)
%time P.simpliciality()

CPU times: user 126 ms, sys: 17 µs, total: 126 ms
Wall time: 126 ms


2

In [17]:
P = polytopes.hypersimplex(12,5)
%time P.simplicity()

CPU times: user 130 ms, sys: 0 ns, total: 130 ms
Wall time: 130 ms


9

## Face lattice and flag-$f$-vector

In [4]:
C = polytopes.permutahedron(7).combinatorial_polyhedron()
%time a = C.face_lattice()

CPU times: user 4.87 s, sys: 32 ms, total: 4.91 s
Wall time: 4.9 s


Used for the flag-$f$-vector:

In [22]:
polytopes.hypercube(4).flag_f_vector(0,3)

64

## Outlook on the development of Polyhedra

### Polytopal construction respect the backend (done)

In [24]:
polytopes.cube(backend='cdd').polar().backend()

'cdd'

### Move combinatorial methods to `CombinatorialPolyhedron` (on-going)

### Add polytopal constructions to `CombinatorialPolyhedron` (on-going)

### Parallel and improved $f$-vector (on-going)

### Lazy double description (future)

### Precomputed data (on-going)

- backends:
  - `field` (done),
  - `normaliz` (almost done),
  - `polymake` (future)
- library:
    - hypercube
    - crosspolytope
    - ...
- constructions
  - dilation
  - translation
  - polar (`sage` 9.2)
  - ...

In [59]:
%time _ = polytopes.hypercube(14, backend='ppl')    # uses Hrepresentation now  # was > 2s
%time _ = polytopes.hypercube(14, backend='field')  # precomputed

CPU times: user 510 ms, sys: 123 µs, total: 510 ms
Wall time: 509 ms
CPU times: user 197 ms, sys: 0 ns, total: 197 ms
Wall time: 197 ms


Pick the best way to compute the dilation:

In [5]:
P = polytopes.hypercube(14, backend='ppl')
%time _ = 2*P  # picks Hrepresentation  # was > 2s
P = polytopes.cross_polytope(14, backend='ppl') 
%time _ = 2*P  # picks Vrepresentation

CPU times: user 505 ms, sys: 0 ns, total: 505 ms
Wall time: 505 ms
CPU times: user 967 ms, sys: 0 ns, total: 967 ms
Wall time: 967 ms


In [19]:
%time P = polytopes.one_hundred_twenty_cell(backend='normaliz')  # set up with Vrepresentation
%time _ = P + P.center()                                         # set up with Hrepresentation

CPU times: user 977 ms, sys: 65.5 ms, total: 1.04 s
Wall time: 483 ms
CPU times: user 297 ms, sys: 16.6 ms, total: 314 ms
Wall time: 182 ms


In [13]:
%time P = P.base_extend(base_ring=P.base_ring(), backend='field')  # changing backend to field in no time
%time _ = P + P.center()  # only trivial computation

CPU times: user 31 µs, sys: 0 ns, total: 31 µs
Wall time: 36.5 µs
CPU times: user 15.3 ms, sys: 0 ns, total: 15.3 ms
Wall time: 15.4 ms


## User input wanted

- Any questions?
- What is missing?
- What is difficult to use?
- What is too slow?

### Give your input now or in one of the following ways:

- modify [trac.sagemath.org/wiki/SagePolyhedralGeometry](https://trac.sagemath.org/wiki/SagePolyhedralGeometry) (optionally open a trac ticket)
- post on [groups.google.com/forum/#!forum/sage-devel](https://groups.google.com/forum/#!forum/sage-devel)
- ask on [ask.sagemath.org](https://ask.sagemath.org)